## dslim/bert-base-NER

In [1]:

# --- 1.1: SETUP E IMPORT (BLOCCO LOCALE) ---
import os
import glob
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification

print("Esecuzione Blocco 1: Difensore Locale con 'transformers'")

# --- 1.2: CARICAMENTO E CONFIGURAZIONE MODELLO TRANSFORMERS ---
print("Caricamento del modello NER 'dslim/bert-base-NER' in corso...")
try:
    tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
    model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")
    # Crea la pipeline per il Named Entity Recognition
    # grouped_entities=True raggruppa token consecutivi (es. "Mario" e "Rossi")
    ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)
    print("✅ Modello 'transformers' caricato con successo!")
except Exception as e:
    print(f"❌ Errore durante il caricamento del modello: {e}")
    print("Assicurati di avere una connessione internet e che le librerie 'transformers' e 'torch' siano installate.")
    exit()

# --- 1.3: FUNZIONE DI ANONIMIZZAZIONE ---
def anonymize_document_with_transformers(text: str) -> str:
    """Usa la pipeline di transformers per trovare e mascherare le entità."""
    # Eseguiamo l'analisi NER
    entities = ner_pipeline(text)
    
    anonymized_text = text
    
    # Ordiniamo le entità dalla più lunga alla più corta per evitare sostituzioni parziali
    entities_to_mask = sorted(entities, key=lambda e: len(e['word']), reverse=True)
    
    for ent in entities_to_mask:
        # Adattiamo le etichette del modello ai placeholder richiesti
        label_map = {
            "PER": "[NOME_PERSONA]",
            "LOC": "[INDIRIZZO]",
            "ORG": "[NOME_ORGANIZZAZIONE]",
            "MISC": "[DATO_GENERICO]"
        }
        # Il modello restituisce 'entity_group' (es. 'PER') e 'word' (es. 'Mario Rossi')
        placeholder = label_map.get(ent['entity_group'], "[DATO_SENSIBILE]")
        anonymized_text = anonymized_text.replace(ent['word'], placeholder)
        
    return anonymized_text

# --- 1.4: ESECUZIONE PRINCIPALE (BLOCCO LOCALE) ---
input_directory = '.'
output_directory = 'output_bert-base-NER'

if not os.path.exists(output_directory):
    os.makedirs(output_directory)
    print(f"✅ Cartella di output '{output_directory}' creata.")

# Cerca tutti i file .txt nella directory di input, escludendo quelli già anonimizzati
all_files = glob.glob(os.path.join(input_directory, '*.txt'))
input_files = [f for f in all_files if 'anon_' not in os.path.basename(f)]

if not input_files:
    print("❌ Nessun file .txt di input trovato.")
else:
    print(f"\n▶️ Trovati {len(input_files)} file da anonimizzare...")
    for file_path in input_files:
        file_name = os.path.basename(file_path)
        print(f"\n--- Elaborazione di: {file_name} ---")
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                original_text = f.read()
            
            anonymized_content = anonymize_document_with_transformers(original_text)
            
            output_file_path = os.path.join(output_directory, f"anon_{file_name}")
            
            with open(output_file_path, 'w', encoding='utf-8') as f:
                f.write(anonymized_content)
                
            print(f"🛡️  File anonimizzato e salvato in: {output_file_path}")

        except Exception as e:
            print(f"❌ Errore durante l'analisi del file {file_name}: {e}")

print("\n--- ✅ Blocco 1: Processo di anonimizzazione completato. ---")

c:\Users\AE653FD\AppData\Local\anaconda3\envs\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Esecuzione Blocco 1: Difensore Locale con 'transformers'
Caricamento del modello NER 'dslim/bert-base-NER' in corso...


c:\Users\AE653FD\AppData\Local\anaconda3\envs\env\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\AE653FD\AppData\Local\anaconda3\envs\env\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
c:\Users\AE653FD\AppData\Local\anaconda3\envs\env\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'huggingface.co'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  w

✅ Modello 'transformers' caricato con successo!

▶️ Trovati 6 file da anonimizzare...

--- Elaborazione di: email_richiesta_supporto.txt ---
🛡️  File anonimizzato e salvato in: output_bert-base-NER\anon_email_richiesta_supporto.txt

--- Elaborazione di: Fattura.txt ---
🛡️  File anonimizzato e salvato in: output_bert-base-NER\anon_Fattura.txt

--- Elaborazione di: Mail.txt ---
🛡️  File anonimizzato e salvato in: output_bert-base-NER\anon_Mail.txt

--- Elaborazione di: nota_di_credito.txt ---
🛡️  File anonimizzato e salvato in: output_bert-base-NER\anon_nota_di_credito.txt

--- Elaborazione di: ordine_di_acquisto.txt ---
🛡️  File anonimizzato e salvato in: output_bert-base-NER\anon_ordine_di_acquisto.txt

--- Elaborazione di: prova.txt ---
🛡️  File anonimizzato e salvato in: output_bert-base-NER\anon_prova.txt

--- ✅ Blocco 1: Processo di anonimizzazione completato. ---


## Azure OpenAI

In [ ]:
# --- 2.1: SETUP E IMPORT (BLOCCO CLOUD) ---
import os
import glob
import json
from openai import AzureOpenAI

print("\nEsecuzione Blocco 2: Analista Cloud")

# --- 2.2: CONFIGURAZIONE AZURE OPENAI ---
# Inserisci qui le tue credenziali e impostazioni.
AZURE_ENDPOINT = "https://fedocana.openai.azure.com/"
AZURE_KEY = "KEY" # NOTA: Inserisci la tua vera chiave qui
AZURE_DEPLOYMENT = "gpt-4o-mini"
API_VERSION = "2024-02-15-preview"

# Cartelle di input e output per questo blocco
ANONYMIZED_INPUT_DIR = 'output_bert-base-NER'
AZURE_OUTPUT_DIR = 'output_azure_openai'

# "Super Prompt" per l'analisi del documento GIA' ANONIMIZZATO
SYSTEM_PROMPT = """
Sei un analista documentale esperto per l'azienda SmartDocs Srl. Riceverai un documento GIA' ANONIMIZZATO, dove i dati sensibili sono stati sostituiti da segnaposto come [NOME_PERSONA], [INDIRIZZO], etc.

Il tuo compito è analizzare il testo e restituire un oggetto JSON con le seguenti chiavi: "tipo_documento", "riassunto", "punti_chiave", e "bozza_risposta".

1.  `tipo_documento`: Classifica il documento scegliendo una delle seguenti opzioni: 'Fattura', 'Email', 'Ordine di Acquisto', 'Nota di Credito', 'Richiesta di Supporto', 'Sconosciuto'.
2.  `riassunto`: Fornisci un breve riassunto (2-3 frasi) dello scopo principale del documento.
3.  `punti_chiave`: Estrai una lista di 3-5 punti chiave o azioni richieste dal documento (es. "Richiesta cambio IBAN", "Sollecito di pagamento per fattura X").
4.  `bozza_risposta`: Scrivi una breve e cortese bozza di risposta per il cliente, confermando la ricezione e indicando che la richiesta è in fase di elaborazione.

Rispondi solo ed esclusivamente con l'oggetto JSON formattato correttamente.
"""

# --- 2.3: ESECUZIONE PRINCIPALE (BLOCCO CLOUD) ---

def process_anonymized_directory():
    """
    Trova tutti i file anonimizzati, li processa con Azure OpenAI,
    stampa l'analisi e salva i risultati in una nuova cartella.
    """
    # Creiamo la cartella di output di Azure se non esiste
    if not os.path.exists(AZURE_OUTPUT_DIR):
        os.makedirs(AZURE_OUTPUT_DIR)
        print(f"✅ Cartella di output '{AZURE_OUTPUT_DIR}' creata.")

    file_list = glob.glob(os.path.join(ANONYMIZED_INPUT_DIR, "anon_*.txt"))
    
    if not file_list:
        print(f"❌ Nessun file anonimizzato trovato nella cartella '{ANONYMIZED_INPUT_DIR}'. Esegui prima il Blocco 1.")
        return

    print(f"\n▶️ Trovati {len(file_list)} documenti anonimizzati da analizzare con il Cloud AI...")

    try:
        client = AzureOpenAI(
            azure_endpoint=AZURE_ENDPOINT,
            api_key=AZURE_KEY,
            api_version=API_VERSION
        )
        print("✅ Client Azure OpenAI configurato.")
    except Exception as e:
        print(f"❌ Errore nella configurazione del client Azure: {e}")
        return

    for file_path in file_list:
        file_name = os.path.basename(file_path)
        print(f"\n\n--- 🧠 Analisi Cloud del file: '{file_name}' ---")
        try:
            with open(file_path, 'r', encoding='utf-8') as file:
                documento_anonimizzato = file.read()

            messages = [
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": documento_anonimizzato}
            ]

            response = client.chat.completions.create(
                model=AZURE_DEPLOYMENT,
                messages=messages,
                temperature=0.1,
                response_format={"type": "json_object"}
            )
            
            risposta_json_str = response.choices[0].message.content
            dati_analizzati = json.loads(risposta_json_str)

            # Stampa i risultati dell'analisi in modo ordinato
            print(f"📄 Tipo Documento: {dati_analizzati.get('tipo_documento', 'N/D')}")
            print(f"\n📝 Riassunto: {dati_analizzati.get('riassunto', 'N/D')}")
            
            print("\n🔑 Punti Chiave:")
            punti = dati_analizzati.get('punti_chiave', [])
            if punti:
                for punto in punti:
                    print(f"   - {punto}")
            else:
                print("   - Nessun punto chiave estratto.")

            print(f"\n✉️ Bozza di Risposta Proposta:\n---\n{dati_analizzati.get('bozza_risposta', 'N/D')}\n---")
            
            # --- NUOVA PARTE: SALVATAGGIO DELL'ANALISI ---
            # Definiamo il percorso del file di output per l'analisi JSON
            output_analysis_path = os.path.join(AZURE_OUTPUT_DIR, f"analysis_{file_name}")
            with open(output_analysis_path, 'w', encoding='utf-8') as f:
                # Usiamo json.dump per salvare il JSON in modo leggibile (indentato)
                json.dump(dati_analizzati, f, indent=4, ensure_ascii=False)
            print(f"💾 Analisi salvata in: {output_analysis_path}")
            # --- FINE NUOVA PARTE ---
            
            print("-" * 50)

        except json.JSONDecodeError:
            print("❌ ERRORE: Il modello non ha restituito un JSON valido.")
        except Exception as e:
            print(f"❌ Si è verificato un errore imprevisto: {e}")

# Esegui la funzione principale del Blocco 2
if __name__ == "__main__":
    # Nello script, questo previene l'esecuzione se importato.
    # In un notebook, si può semplicemente eseguire la cella.
    # Per simulare l'esecuzione separata, chiamiamo esplicitamente la funzione.
    
    # NOTA: Assicurati di aver eseguito il Blocco 1 prima di questo.
    process_anonymized_directory()

print("\n--- ✅ Blocco 2: Processo di analisi cloud completato. ---")


Esecuzione Blocco 2: Analista Cloud

▶️ Trovati 6 documenti anonimizzati da analizzare con il Cloud AI...
✅ Client Azure OpenAI configurato.


--- 🧠 Analisi Cloud del file: 'anon_email_richiesta_supporto.txt' ---
📄 Tipo Documento: Email

📝 Riassunto: Il cliente, [NOME_PER[NOME_ORGANIZZAZIONE]ONA], contatta il supporto di SmartDocs Srl per richiedere un aggiornamento delle coordinate bancarie relative alla fattura N. 734/2025. Viene fornito un nuovo IBAN e richiesto di disattivare quello precedente.

🔑 Punti Chiave:
   - Richiesta di aggiornamento coordinate bancarie
   - Disattivazione dell'IBAN IT60X0542811101000000123456
   - Nuovo IBAN fornito: IT12Y0306909606100000012345
   - Contatto telefonico fornito: 333 1234567

✉️ Bozza di Risposta Proposta:
---
Gentile [NOME_PER[NOME_ORGANIZZAZIONE]ONA],

La ringraziamo per averci contattato. Abbiamo ricevuto la sua richiesta di aggiornamento delle coordinate bancarie e stiamo procedendo con l'elaborazione. La informeremo non appena l'opera